In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from typing import Optional, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool, Tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# Define the tools for the agent to use
@tool
def send_email(recipient_email: Optional[str] = None, subject: Optional[str] = None, body: Optional[str] = None):
    """Send an email to recipient_email with subject and body."""
    missing_details = []
    if not recipient_email:
        missing_details.append("recipient email")
    if not subject:
        missing_details.append("subject")
    if not body:
        missing_details.append("body")
    
    if missing_details:
        return f"Missing details: {', '.join(missing_details)}. Please provide them."
    
    print(f"Sending email to {recipient_email}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    return "Email sent successfully."

tools = [
    send_email
]

tool_node = ToolNode(tools)

model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

# Define the state graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()

def run_assistant(app):
    state = {"messages": []}
    
    while True:
        user_input = input("You: ")
        state["messages"].append(HumanMessage(content=user_input))
        final_state = app.invoke(
            {"messages": state["messages"]},
            config={"configurable": {"thread_id": 53}}
        )
        
        response_content = final_state["messages"][-1].content
        print(f"AI: {response_content}")
        
        # Check if the response indicates missing details
        if "please provide" in response_content:
            # The assistant will ask for missing information
            additional_info = input("You (additional info): ")
            state["messages"].append(HumanMessage(content=additional_info))
        else:
            break

# Run the assistant
workflow_app = workflow.compile(checkpointer=checkpointer)
run_assistant(workflow_app)


AI: Certainly! I'd be happy to help you write an email to vva2@tamu.edu. To send the email, we'll use the `send_email` function. However, we need a subject and body for the email. Since you haven't provided these details, I'll ask you for more information before we proceed.

Could you please provide:
1. The subject of the email
2. The content you'd like in the body of the email

Once you provide this information, I'll be able to draft the email and send it using the available tool.
Sending email to vva2@tamu.edu
Subject: Approaching Deadline for CSCE 689
Body: Dear Professor,

I hope this email finds you well. I am writing to inquire about the approaching deadline for CSCE 689. I wanted to confirm the exact due date and ensure that I have all the necessary information to complete the assignment on time.

Could you please provide any additional details or reminders regarding the upcoming deadline? If there are any specific requirements or guidelines I should be aware of, I would greatly

ValueError: Node `prompt_for_info` is not reachable